In [1]:
from pathlib import Path

In [2]:
def print_dir(input_folder):
    input_path = Path('F:/') / input_folder
    folder_list = list(input_path.iterdir())
    for folder_path in folder_list:
        if folder_path.is_dir():
            file_list = list(folder_path.glob('*.mp4'))
        else:
            file_list = [folder_path]
        
        for file_path in file_list:
            print(file_path.stem)

In [3]:
input_folder = '_Anime Download'
print_dir(input_folder)

[SHIGURE][zeronosyo][01][CHS][720p]
[SHIGURE][zeronosyo][02][CHS][720p]
[SHIGURE][zeronosyo][03][CHS][720p]
[SHIGURE][zeronosyo][04][CHS][720p]
[SHIGURE][zeronosyo][05][CHS][720p]
[SHIGURE][zeronosyo][06][CHS][720p]
[SHIGURE][zeronosyo][07][CHS][720p]
[SHIGURE][zeronosyo][08][CHS][720p]
[SHIGURE][zeronosyo][09][CHS][720p]
[SHIGURE][zeronosyo][10][CHS][720p]
[SHIGURE][zeronosyo][11][CHS][720p]
[SHIGURE][zeronosyo][12][CHS][720p]
[UchiMusume][01][BIG5][1080P]
[UchiMusume][02][BIG5][1080P]
[UchiMusume][03][BIG5][1080P]
[UchiMusume][04][BIG5][1080P]
[UchiMusume][05][BIG5][1080P]
[UchiMusume][06][BIG5][1080P]
[UchiMusume][07][BIG5][1080P]
[UchiMusume][08][BIG5][1080P]
[UchiMusume][09][BIG5][1080P]
[UchiMusume][10][BIG5][1080P]
[UchiMusume][11][BIG5][1080P]
[UchiMusume][12][BIG5][1080P][END]
[JYFanSub][Absolute Demonic Front Babylonia][01][GB_CN][1080P][HEVC]
[JYFanSub][Absolute Demonic Front Babylonia][02][GB_CN][1080P][HEVC]
[JYFanSub][Absolute Demonic Front Babylonia][03][GB_CN][1080P][HE

In [4]:
input_folder = 'Download - TV Shows (Anime)'
print_dir(input_folder)

[ANi] 進擊的巨人 The Final Season 完結篇 - 01 [1080P][Baha][WEB-DL][AAC AVC][CHT]
[ANi] 進擊的巨人 The Final Season 完結篇 後篇 - 01 [1080P][Baha][WEB-DL][AAC AVC][CHT]
[Comicat&KissSub][Beast Tamer][01v2][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][02v2][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][03][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][04][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][05][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][06][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][07][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][08][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][09v2][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][10][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][11][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][12][1080P][GB][MP4]
[Comicat&KissSub][Beast Tamer][13][1080P][GB][MP4]
[Itai No Wa][01][BIG5][1080P]
[Itai No Wa][02][BIG5][1080P]
[Itai No Wa][03][BIG5][1080P]
[Itai No Wa][04][BIG5][1080P]
[Itai No Wa][05][BIG5][1080P]
[Itai No Wa][06][BIG5][1080P]


In [5]:
input_folder = 'Watch - TV Shows (Anime)'
print_dir(input_folder)

[Sakurato] 86—Eitishikkusu— [01v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [02v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [03v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [04][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [05v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [06][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [07][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [08][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [09][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [10][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [11.5][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [11][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [12v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [13v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikkusu— [14v2][AVC-8bit 1080p@60FPS AAC][CHS]
[Sakurato] 86—Eitishikku